# Optical spectra from time-propagation
[Link to tutorial](https://www.octopus-code.org/documentation/13/tutorial/response/optical_spectra_from_time-propagation/)

In this tutorial we will learn how to obtain the absorption spectrum of a molecule from the explicit solution of the time-dependent Kohn-Sham equations. We choose as a test case methane (CH<sub>4</sub>).


In [ ]:
import pandas as pd

In [ ]:
!mkdir -p 1_Optical_spectra_from_time_propagation

In [ ]:
cd 1_Optical_spectra_from_time_propagation

## Ground state

Before starting out time-dependent simulations, we need to obtain the ground state of the system. For this we use basically the same inp file as in the
[total energy convergence tutorial](../1-basics/3-Total_energy_convergence.ipynb):


In [ ]:
%%writefile inp

stdout = 'stdout_gs.txt'
stderr = 'stderr_gs.txt' 

CalculationMode = gs
UnitsOutput = eV_angstrom

Radius = 3.5*angstrom
Spacing = 0.18*angstrom

CH = 1.097*angstrom
%Coordinates
 "C" |           0 |          0 |           0
 "H" |  CH/sqrt(3) | CH/sqrt(3) |  CH/sqrt(3)
 "H" | -CH/sqrt(3) |-CH/sqrt(3) |  CH/sqrt(3)
 "H" |  CH/sqrt(3) |-CH/sqrt(3) | -CH/sqrt(3)
 "H" | -CH/sqrt(3) | CH/sqrt(3) | -CH/sqrt(3)
%

In [ ]:
!octopus

After running ```octopus```, we will have the Kohn-Sham wave-functions of the ground-state in the directory ```restart/gs```. As we are going to propagate these wave-functions, they have to be well converged. It is important not only to converge the energy (that is relatively easy to converge), but the density.
The default [ConvRelDens](https://www.octopus-code.org/documentation//13/variables/scf/convergence/convreldens) = 1e-06 is usually enough, though.

## Time-dependent run
To calculate absorption, we excite the system with an infinitesimal electric-field pulse, and then propagate the time-dependent Kohn-Sham equations for a certain time ''T''. The spectrum can then be evaluated from the time-dependent dipole moment.

#### Input

This is how the input file should look for the time propagation. It is similar to the one from the [Time-dependent propagation tutorial](../1-basics/6-TD_propagation.ipynb).


In [ ]:
%%writefile inp

stdout = 'stdout_td.txt'
stderr = 'stderr_td.txt'

CalculationMode = td
FromScratch = yes
UnitsOutput = eV_angstrom

Radius = 3.5*angstrom
Spacing = 0.18*angstrom

CH = 1.097*angstrom
%Coordinates
 "C" |           0 |          0 |           0
 "H" |  CH/sqrt(3) | CH/sqrt(3) |  CH/sqrt(3)
 "H" | -CH/sqrt(3) |-CH/sqrt(3) |  CH/sqrt(3)
 "H" |  CH/sqrt(3) |-CH/sqrt(3) | -CH/sqrt(3)
 "H" | -CH/sqrt(3) | CH/sqrt(3) | -CH/sqrt(3)
%

TDPropagator = aetrs
TDTimeStep = 0.0023/eV
TDMaxSteps = 4350  # ~ 10.0/TDTimeStep

TDDeltaStrength = 0.01/angstrom
TDPolarizationDirection = 1

Besides changing the [CalculationMode]() to ```td```, we have added [FromScratch]() = ```yes```. This will be useful if you decide to run the propagation for other polarization directions (see bellow). For the time-evolution we use again the Approximately Enforced Time-Reversal Symmetry (aetrs) propagator. The time-step is chosen such that the propagation remains numerically stable. You should have learned how to set it up in the tutorial [time-dependent propagation tutorial](../1-basics/6-TD_propagation.ipynb). Finally, we set the number of time steps with the variable [TDMaxSteps](https://www.octopus-code.org/documentation//13/variables/time-dependent/propagation/tdmaxsteps). To have a maximum propagation time of 10 $\hbar/{\rm eV}$ we will need around 4350 iterations.

We have also introduced two new input variables to define our perturbation:

* [TDDeltaStrength](https://www.octopus-code.org/documentation//13/variables/time-dependent/response/tddeltastrength): this is the strength of the perturbation. This number should be small to keep the response linear, but should be sufficiently large to avoid numerical problems.

* [TDPolarizationDirection](https://www.octopus-code.org/documentation//13/variables/time-dependent/response/dipole/tdpolarizationdirection): this variable sets the polarization of our perturbation to be on the first axis (''x'').

Note that you will be calculating the singlet dipole spectrum. You can also obtain the triplet by using [TDDeltaStrengthMode](https://www.octopus-code.org/documentation//13/variables/time-dependent/response/tddeltastrengthmode), and other multipole responses by using [TDKickFunction](https://www.octopus-code.org/documentation//13/variables/time-dependent/response/tdkickfunction). For details on triplet calculations see the [Triplet excitations tutorial](../2-Optical-Response/5_Triplet_excitations.ipynb).

You can now start ```Octopus``` and go for a quick coffee (this should take a few minutes depending on your machine). Propagations are slow, but the good news is that they scale very well with the size of the system. This means that even if methane is very slow, a molecule with 200 atoms can still be calculated without big problems.

In [ ]:
!octopus

In [ ]:
!mv td.general/multipoles td.general/multipoles.1

In [ ]:
%%writefile inp

stdout = 'stdout_td.txt'
stderr = 'stderr_td.txt'

CalculationMode = td
FromScratch = yes
UnitsOutput = eV_angstrom

Radius = 3.5*angstrom
Spacing = 0.18*angstrom

CH = 1.097*angstrom
%Coordinates
 "C" |           0 |          0 |           0
 "H" |  CH/sqrt(3) | CH/sqrt(3) |  CH/sqrt(3)
 "H" | -CH/sqrt(3) |-CH/sqrt(3) |  CH/sqrt(3)
 "H" |  CH/sqrt(3) |-CH/sqrt(3) | -CH/sqrt(3)
 "H" | -CH/sqrt(3) | CH/sqrt(3) | -CH/sqrt(3)
%

TDPropagator = aetrs
TDTimeStep = 0.0023/eV
TDMaxSteps = 4350  # ~ 10.0/TDTimeStep

TDDeltaStrength = 0.01/angstrom
TDPolarizationDirection = 2

In [ ]:
!octopus

In [ ]:
!mv td.general/multipoles td.general/multipoles.2

In [ ]:
%%writefile inp

stdout = 'stdout_td.txt'
stderr = 'stderr_td.txt'

CalculationMode = td
FromScratch = yes
UnitsOutput = eV_angstrom

Radius = 3.5*angstrom
Spacing = 0.18*angstrom

CH = 1.097*angstrom
%Coordinates
 "C" |           0 |          0 |           0
 "H" |  CH/sqrt(3) | CH/sqrt(3) |  CH/sqrt(3)
 "H" | -CH/sqrt(3) |-CH/sqrt(3) |  CH/sqrt(3)
 "H" |  CH/sqrt(3) |-CH/sqrt(3) | -CH/sqrt(3)
 "H" | -CH/sqrt(3) | CH/sqrt(3) | -CH/sqrt(3)
%

TDPropagator = aetrs
TDTimeStep = 0.0023/eV
TDMaxSteps = 4350  # ~ 10.0/TDTimeStep

TDDeltaStrength = 0.01/angstrom
TDPolarizationDirection = 3

In [ ]:
!octopus

In [ ]:
!mv td.general/multipoles td.general/multipoles.3

## Output
The output should be very similar to the one from the [time-dependent propagation tutorial](../1-basics/6-TD_propagation.ipynb). The main difference is the information about the perturbation:


In [ ]:
!grep -am 1 -A 2 "Applying delta kick" stdout_td.txt

In [ ]:
!head -n 20 td.general/multipoles.1

Note how the dipole along the ''x'' direction (forth column) changes in response to the perturbation.

##  Optical spectra

In order to obtain the spectrum for a general system one would need to perform three time-dependent runs, each for a perturbation along a different Cartesian direction (''x'', ''y'', and ''z''). In practice this is what you would need to do:
- Set the direction of the perturbation along ''x'' ([TDPolarizationDirection](https://www.octopus-code.org/documentation//13/variables/time-dependent/response/dipole/tdpolarizationdirection) = 1),
- Run the time-propagation,
- Rename the ```td.general/multipoles``` file to ```td.general/multipoles.1```,
- Repeat the above step for directions ''y'' ([TDPolarizationDirection](https://www.octopus-code.org/documentation//13/variables/time-dependent/response/dipole/tdpolarizationdirection)) and ''z'' ([TDPolarizationDirection](https://www.octopus-code.org/documentation//13/variables/time-dependent/response/dipole/tdpolarizationdirection) = 3) to obtain files ```td.general/multipoles.2``` and ```td.general/multipoles.3```.

Nevertheless, the $T_d$ symmetry of methane means that the response is identical for all directions and the absorption spectrum for ''x''-polarization will in fact be equivalent to the spectrum averaged over the three directions. You can perform the calculations for the ''y'' and ''z'' directions if you need to convince yourself that they are indeed equivalent, but you this will not be necessary to complete this tutorial.

Note that ```octopus``` can actually use the knowledge of the symmetries of the system when calculating the spectrum. However, this is fairly complicated to understand for the purposes of this introductory tutorial, so it will be covered in the [Use of symmetries in optical spectra from time-propagation tutorial](../2-Optical-Response/6_Use_of_symmetries_in_optical_spectra_from_time_propagation.ipynb).

### The ```oct-propagation_spectrum``` utility

```octopus``` provides an utility called [oct-propagation_spectrum](https://www.octopus-code.org/documentation//13/manual/utilities/oct-propagation_spectrum) to process the ```multipoles``` files and obtain the spectrum.

#### Input
This utility requires little information from the input file, as most of what it needs is provided in the header of the ```multipoles``` files. If you want you can reuse the same input file as for the time-propagation run, but the following input file will also work:

```
%%writefele inp
UnitsOutput = eV_angstrom
```

Don't worry about the warnings generated, we know what we are doing!

In [ ]:
!oct-propagation_spectrum

As we used the full input file, we see some more parser warnings, which indicate the unused variables.
Here, these arise as the utility does not need them, but warnings like this also could indicate a mis-typed variable name.


In [ ]:
!cat stdout_td.txt

You will notice that the file ```cross_section_vector``` is created. If you have all the required information (either a symmetric molecule and one multipole file, or a less symmetric molecule and multiple multipole files), ```oct-propagation_spectrum``` will generate the whole polarizability tensor, ```cross_section_tensor```. If not enough multipole files are available, it can only generate the response to the particular polarization you chose in you time-dependent run, ```cross_section_vector```.

### Cross-section vector

Let us look first at ```cross_section_vector```:


In [ ]:
!head -n 12 cross_section_vector.1

The beginning of the file just repeats some information concerning the run.


In [ ]:
!grep -am 1 -B 1 -A 9 "#%" cross_section_vector.1

Now comes a summary of the variables used to calculate the spectra. Note that you can change all these settings just by adding these variables to the input file before running ```oct-propagation_spectrum```.  Of special importance are perhaps [PropagationSpectrumMaxEnergy](https://www.octopus-code.org/documentation//13/variables/utilities/oct-propagation_spectrum/propagationspectrummaxenergy) that sets the maximum energy that will be calculated, and [PropagationSpectrumEnergyStep](https://www.octopus-code.org/documentation//13/variables/utilities/oct-propagation_spectrum/propagationspectrumenergystep) that determines how many points your spectrum will contain. To have smoother curves, you should reduce this last variable.


In [ ]:
!grep -am 1 -B 2 -A 2 "Electronic sum rule" cross_section_vector.1

Now comes some information from the sum rules. The first is just the ''f''-sum rule, which should yield the number of active electrons in the calculations. We have 8 valence electrons (4 from carbon and 4 from hydrogen), but the sum rule gives a number that is much smaller than 8! The reason is that we are just summing our spectrum up to 20 eV (see [PropagationSpectrumMaxEnergy](https://www.octopus-code.org/documentation//13/variables/utilities/oct-propagation_spectrum/propagationspectrummaxenergy), but for the sum rule to be fulfilled, we should go to infinity. Of course infinity is a bit too large, but increasing 20 eV to a somewhat larger number will improve dramatically the ''f''-sum rule. The second number is the static polarizability also calculated from the sum rule. Again, do not forget to converge this number with [PropagationSpectrumEnergyStep](https://www.octopus-code.org/documentation//13/variables/utilities/oct-propagation_spectrum/propagationspectrumenergystep).


In [ ]:
df = pd.read_csv(
    "cross_section_vector.1",
    delimiter="     ",
    usecols=[0, 4],
    names=["Energy (eV)", "StrengthFunction"],
    skiprows=26,
    engine="python",
)
df

Finally comes the spectrum. The first column is the energy (frequency), the next three columns are a row of the cross-section tensor, and the last one is the strength function for this run.

The dynamic polarizability is related to optical absorption cross-section via $\sigma \left( \omega \right) = \frac{4 \pi \omega}{c} \mathrm{Im}\ \alpha \left( \omega \right) $ in atomic units, or more generally $4 \pi \omega \tilde{\alpha}\ \mathrm{Im}\ \alpha \left( \omega \right) $ (where $\tilde{\alpha}$ is the fine-structure constant) or $\frac{\omega e^2}{\epsilon_0 c} \mathrm{Im}\ \alpha \left( \omega \right) $. The cross-section is related to the strength function by $S \left( \omega \right) = \frac{mc}{2 \pi^2 \hbar^2} \sigma \left( \omega \right)$.

## Cross-section tensor

If all three directions were done, we would have four files: ```cross_section_vector_1``` , ```cross_section_vector_2``` , ```cross_section_vector_3``` , and ```cross_section_tensor``` . The latter would be similar to:

In [ ]:
!head -n 20 cross_section_tensor

In [ ]:
df.plot(
    x="Energy (eV)",
    y="StrengthFunction",
    legend=False,
    title="Absorption spectrum of methane",
    ylabel="Strength function (1/eV)",
);


The columns are now the energy, the average absorption coefficient (Tr $\sigma/3$), the anisotropy, and
then all the 9 components of the tensor. The third number is the spin component, just 1 here since it is unpolarized. The anisotropy is defined as

$
  (\Delta \sigma)^2 = \frac{1}{3} \{ 3\rm Tr (\sigma ^2) - \rm [Tr(\sigma)]^2 \}
$

The reason for this definition is that it is identically equal to:

$
 (\Delta \sigma)^2 = (1/3) [ (\sigma_1-\sigma_2)^2 + (\sigma_1-\sigma_3)^2 + (\sigma_2-\sigma_3)^2 ],
$

where $\{\sigma_1, \sigma_2, \sigma_3\}\,$ are the eigenvalues of $\sigma\,$. An "isotropic" tensor is characterized by having three equal eigenvalues, which leads to zero anisotropy. The more different that the eigenvalues are, the larger the anisotropy is.

If you now plot the absorption spectrum (column 5 vs 1 in ```cross_section_vector```, but you can use ```cross_section_tensor``` for this exercise in case you did propagate in all three directions), you should obtain the plot shown above. Of course, you should now try to converge this spectrum with respect to the calculation parameters. In particular:

* Increasing the total propagation time will reduce the width of the peaks. In fact, the width of the peaks in this methods is absolutely artificial, and is inversely proportional to the total propagation time. Do not forget, however, that the area under the peaks has a physical meaning: it is the oscillator strength of the transition.
* A convergence study with respect to the spacing and box size might be necessary. This is covered in the next tutorial.

Some questions to think about:
* What is the equation for the peak width in terms of the propagation time? How does the observed width compare to your expectation?
* What is the highest energy excitation obtainable with the calculation parameters here? Which is the key one controlling the maximum energy?
* Why does the spectrum go below zero? Is this physical? What calculation parameters might change this?





[Go to *2_Convergence_of_optical_spectra.ipynb*](2_Convergence_of_optical_spectra.ipynb)